In [28]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from geopy.distance import geodesic

spark = SparkSession.builder.getOrCreate()

In [38]:
from pyspark.sql.types import FloatType

@F.udf(returnType=FloatType())
def geodesic_udf(a_lat, a_long, b_lat, b_long):
    return geodesic((a_lat, a_long), (b_lat, b_long)).miles

In [52]:
df = spark.read.csv('country-capitals.csv', header=True)
df = df.withColumn("idx", F.monotonically_increasing_id())

df = df.alias('left').join(df.alias('right'), F.col('left.idx') > F.col('right.idx')).select(
    F.col('left.CountryName'), 
    F.col('left.CapitalName'), 
    F.col('right.CountryName'), 
    F.col('right.CapitalName'),
    geodesic_udf(F.col('left.CapitalLatitude'), F.col('left.CapitalLongitude'), F.col('right.CapitalLatitude'), F.col('right.CapitalLongitude')).alias('distance')
).orderBy(F.col('distance'))
df.show(100)

+--------------------+----------------+--------------------+----------------+----------+
|         CountryName|     CapitalName|         CountryName|     CapitalName|  distance|
+--------------------+----------------+--------------------+----------------+----------+
|           Hong Kong|             N/A|          Antarctica|             N/A|       0.0|
|               Macau|             N/A|           Hong Kong|             N/A|       0.0|
|               Macau|             N/A|Heard Island and ...|             N/A|       0.0|
|US Minor Outlying...|Washington, D.C.|       United States|Washington, D.C.|       0.0|
|Heard Island and ...|             N/A|          Antarctica|             N/A|       0.0|
|Heard Island and ...|             N/A|           Hong Kong|             N/A|       0.0|
|              Israel|       Jerusalem|           Palestine|       Jerusalem|       0.0|
|               Macau|             N/A|          Antarctica|             N/A|       0.0|
|     Northern Cyprus

22/10/20 14:50:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 876634 ms exceeds timeout 120000 ms
22/10/20 14:50:45 WARN SparkContext: Killing executors is not supported by current scheduler.
